In [12]:
from bs4 import BeautifulSoup
import urllib.request as REQ 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import folium  # 지도 api를 그려주는 라이브러리
import os
import sys
import urllib.request
import json

# 글꼴을 바꿔라
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
# 유니코드를 적용해라
matplotlib.rcParams['axes.unicode_minus'] = False

In [6]:
df = pd.read_excel('menu.xlsx')
df

,랭킹,상호명,업종,지역,전화번호,주소
0,1,아카사카,일식,서초구 서초동,(02) 585-1145,서울 서초구 서초동 1327 B-2호(서초타운 트라팰리스)
1,2,룰라빈대떡,카페/주점,서초구 서초동,(02) 585-7870,서울 서초구 서초동 1327-13
2,3,홍대돈부리 강남역점,일식,강남구 역삼동,(02) 565-8064,서울 강남구 역삼동 810-8
3,4,코이카페(KOI cafe),카페/주점,강남구 역삼동,(02) 561-8511,서울 강남구 역삼동 825-25
4,5,삼호복집,한식,서초구 서초2동,(02) 3474-3533,서울 서초구 서초2동 1338-10
5,6,일마레(IL MARE),양식,강남구 역삼동,(02) 501-2490,서울 강남구 역삼동 817-37
6,7,스시효 서초점,일식,서초구 서초동,(02) 521-3593,서울 서초구 서초동 1327
7,8,마노디셰프 강남점,양식,강남구 역삼동,(02) 553-5311,서울 강남구 역삼동 825-18 캠브리지빌딩 지하 1층
8,9,하남돼지집 강남대로점,한식,강남구 역삼동,(02) 501-5292,서울 강남구 역삼동 831-29
9,10,화덕생선구이 강남점,한식,강남구 역삼동,(02) 562-4381,서울 강남구 역삼동 823 풍림빌딩


In [19]:
# 37°33'58.87"N 126°58'40.63"E
# zoom_start - 커질수록 더 커진다.
map = folium.Map(location=[37.5012731,127.0396276],zoom_start=16)

m = folium.Marker(location=[37.5012731,127.0396276],popup="multicampus")
m.add_to(map)
map

In [13]:
# 주소를 위도 경도로 반환해주는 함수
def addrToLatLng(s):
    client_id = "ppihLuN2pr20zbkL0CAJ"
    client_secret = "OJqch1hs7D"
    encText = urllib.parse.quote(s)
    url = "https://openapi.naver.com/v1/map/geocode?query=" + encText # json 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        aa= response_body.decode('utf-8') 
        bb = json.loads(aa)
        return bb['result']['items'][0]['point']
    else:
        return "Error Code:" + rescode

In [14]:
addrToLatLng('강남구 테헤란로 212')

{'x': 127.0396276, 'y': 37.5012731}

In [29]:
# zip = 각각의 리스트를 인덱스별로 튜플로 묶어서 리스트속에다가 넣어줌
a = ['aa','bb','cc']
b = [10,20,30]
c = [1,2,3]
z = zip(a,b,c)
# print(list(z))
for n1,n2,n3 in z:
    print(n1,n2,n3)

aa 10 1
bb 20 2
cc 30 3


In [31]:
map = folium.Map(location=[37.5012731,127.0396276],zoom_start=14)

for a,b in zip(df['상호명'], df['주소']):
#     print(a,b)
    d = addrToLatLng(b)
    m = folium.Marker(location=[d['y'],d['x']],popup=a)
    m.add_to(map)
map

In [37]:
lat = []
lng = []
for addr in df['주소']:
    d = addrToLatLng(addr)
    print(d)
    lat.append(d['y'])
    lng.append(d['x'])

{'x': 127.0270925, 'y': 37.4956378}
{'x': 127.0281203, 'y': 37.4947968}
{'x': 127.0262372, 'y': 37.5029952}
{'x': 127.0299598, 'y': 37.4969761}
{'x': 127.0298623, 'y': 37.4911738}
{'x': 127.0284867, 'y': 37.5008427}
{'x': 127.0270925, 'y': 37.4956378}
{'x': 127.0294029, 'y': 37.4980922}
{'x': 127.0307645, 'y': 37.4947543}
{'x': 127.0315897, 'y': 37.4987085}
{'x': 127.0302824, 'y': 37.4887898}
{'x': 127.0289609, 'y': 37.4916564}
{'x': 127.0280603, 'y': 37.500122}
{'x': 127.0265545, 'y': 37.5028692}
{'x': 127.0283547, 'y': 37.4940113}
{'x': 127.0287828, 'y': 37.4996935}
{'x': 127.0286563, 'y': 37.4999187}
{'x': 127.0217384, 'y': 37.5039614}
{'x': 127.0248115, 'y': 37.5015912}
{'x': 127.0320137, 'y': 37.495352}
{'x': 127.0254253, 'y': 37.4975868}
{'x': 127.0258475, 'y': 37.4986004}
{'x': 127.0271279, 'y': 37.5024419}
{'x': 127.0323577, 'y': 37.4970756}
{'x': 127.0258331, 'y': 37.4993986}


In [42]:
latMean = sum(lat)/len(lat)
lngMean = sum(lng)/len(lng)
map = folium.Map(location=[latMean,lngMean],zoom_start=14)

for store,la,ln in zip(df['상호명'],lat,lng):
    m = folium.Marker([la,ln],popup=store)
    m.add_to(map)
map